# BUILDING WEBSITE-AGENT IN 5 SIMPLE STEPS

### STEP 1: SETTING UP THE ENVIRONMENT

In [1]:
# STEP 1: SET UP THE ENVIRONMENT

# STEP 1.1 INSTALL THE REQUIRED PACKAGES
!pip install langchain_community
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.2 MB/s eta 0:00:00


In [2]:
# STEP 1.2 IMPORT THE NECESSARY MODULES

# For loading the OpenAI API key safely into the colab environment
import os
from google.colab import userdata

# For Extracting the text from the URL
import requests
from bs4 import BeautifulSoup

# For creating the agent
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI

In [8]:
# STEP 1.3 LOAD THE OPEN AI API KEY
os.environ["OPENAI_API_KEY"] = userdata.get('sep_openai')

### STEP 2: EXTRACTING TEXT FROM THE URL

In [9]:
# STEP 2: DEFINE A FUNCTION TO EXTRACT TEXT FROM URL
def extract_website_text(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  return soup.get_text()






### STEP 3: CREATING THE WEBSITE Q AND A TOOL FOR THE AGENT

In [10]:
# STEP 3 CREATE THE TOOL


# STEP 3.1 DEFINE THE FUNCTION FOR PROCESSING THE USER INPUT
def website_qa_tool(url_and_question):
        url, question = url_and_question.split("|", 1)
        content = extract_website_text(url.strip())
        prompt = f"Answer this question using the content from the website:\n\n{question.strip()}\n\nContent:\n{content[:6000]}"
        llm = ChatOpenAI()
        return llm.predict(prompt)


# STEP 3.2 CREATE THE TOOL FOR THE AGENT USING THE FUNCTION CREATED

tool = Tool(
    name="WebsiteQnA",
    func=website_qa_tool,
    description="Input format: '<website URL> | <your question>'. Use this tool to ask questions about any website's content."

)


### STEP 4: INITIALIZING THE AGENT

In [11]:
# STEP 4: INITIALIZE THE AGENT

agent = initialize_agent(
    tools=[tool],
    llm=ChatOpenAI(),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)


### STEP 5: RUNNING THE AGENT & CHECKING THE MAGIC!

In [14]:
# STEP 5: RUN THE AGENT
print(agent.run("https://github.com/Premalatha-success | How many followers are there?"))




> Entering new AgentExecutor chain...
I should use WebsiteQnA to find out how many followers are on the GitHub page.
Action: WebsiteQnA
Action Input: https://github.com/Premalatha-success | How many followers are there?
Observation: Premalatha T has 2.1k followers.
Thought:I now know the final answer.
Final Answer: Premalatha T has 2.1k followers on GitHub.

> Finished chain.
Premalatha T has 2.1k followers on GitHub.
